In [0]:
import pandas as pd
import numpy as np

In [0]:
#Function receives a list of results and counts the number of seats for each party.
#Results are returned as a dictionary.
#Does not work for Green and Independent.

def countResults(results):
  liberal = 0
  conservative = 0
  ndp = 0
  bloc = 0
  for i in range(0, len(results)-1):
    if results[i] == 'Conservative':
      conservative +=1
    elif results[i] == 'Liberal':
      liberal += 1
    elif results[i] == 'NDP':
      ndp += 1 
    elif results[i] == 'Bloc':
      bloc += 1
  
  counted = {'Liberal': liberal, 'Conservative': conservative,
            'NDP': ndp, 'Bloc': bloc}
  
  return counted
      
    
  

# Read the datafiles

In [3]:




df_results = pd.read_pickle("../Census_Election_Results.pkl")

print(df_results.shape)

df_results.tail(20)




(1564, 11)


,"Population, 2016",Population density per square kilometre,Average age of the population,Average household size,Eng,Fr,Non-of,Average total income in 2015 among recipients ($),PovertyRate,Im,Results
1544,115340.0,4386.0,41.3,2.3,0.536386,0.009569,0.454044,41676.0,16.4,0.440317,Conservative
1545,124605.0,7.7,46.7,2.3,0.906348,0.012648,0.081004,41500.0,14.6,0.104771,Liberal
1546,115344.0,344.1,41.9,2.4,0.692074,0.013940,0.293986,58194.0,12.9,0.358098,Conservative
1547,101101.0,49.4,40.3,2.7,0.833595,0.011665,0.154740,47281.0,9.3,0.196608,Liberal
1548,110817.0,1163.7,40.3,2.6,0.618331,0.011737,0.369932,48514.0,13.2,0.368291,Conservative
1549,110995.0,0.4,37.7,2.5,0.893568,0.012551,0.093881,54071.0,9.1,0.080971,NDP
1550,98396.0,2276.0,42.7,2.6,0.317862,0.005110,0.677028,37803.0,24.8,0.652441,Conservative
1551,120834.0,367.5,42.2,2.3,0.860515,0.020387,0.119098,45081.0,11.6,0.159273,NDP
1552,107339.0,214.9,48.1,2.3,0.835844,0.012444,0.151712,49898.0,11.7,0.220135,Conservative
1553,114695.0,6.5,48.2,2.1,0.869509,0.016696,0.113795,40866.0,15.8,0.132135,Liberal


# New Section

In [4]:

#Creating the results
y = df_results["Results"]

df_X = df_results.drop(columns = ['Results'])
print(df_X.columns)

df_X.head()


Index(['Population, 2016', 'Population density per square kilometre',
       'Average age of the population', 'Average household size', 'Eng', 'Fr',
       'Non-of', '    Average total income in 2015 among recipients ($)',
       'PovertyRate', 'Im'],
      dtype='object')


,"Population, 2016",Population density per square kilometre,Average age of the population,Average household size,Eng,Fr,Non-of,Average total income in 2015 among recipients ($),PovertyRate,Im
0,86494.0,13.4,42.4,2.4,0.992109,0.002280,0.005611,48083.0,9.3,0.011971
1,74116.0,4.4,47.4,2.3,0.992590,0.002379,0.005031,38924.0,12.6,0.009546
2,77680.0,2.0,46.5,2.3,0.991748,0.001949,0.006302,38258.0,15,0.009691
3,27197.0,0.1,37.0,2.7,0.864492,0.015284,0.120224,56347.0,6.5,0.030252
4,86553.0,2.3,46.5,2.3,0.986168,0.006595,0.007237,38151.0,14.9,0.014009


In [5]:
#Preprocessing done before the Train Test Split 

import sklearn.preprocessing as skprep
scaler = skprep.MinMaxScaler()
print(scaler)
scaler.fit(df_X)
X = scaler.transform(df_X)



MinMaxScaler(copy=True, feature_range=(0, 1))


In [6]:
print(X.shape[0])

1564


# Create a Train Test Split

y values have been determined above

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [8]:
X_train

array([[4.23779190e-01, 1.38825633e-02, 5.93301435e-01, ...,
        3.52556429e-01, 1.82456140e-01, 2.30326453e-01],
       [5.96045670e-01, 1.47181036e-01, 6.26794258e-01, ...,
        4.86665597e-01, 6.35087719e-01, 5.82673289e-01],
       [7.32630443e-01, 1.16109734e-02, 5.64593301e-01, ...,
        1.89951994e-01, 3.96491228e-01, 6.17091619e-01],
       ...,
       [6.05836475e-01, 2.02418906e-04, 5.64593301e-01, ...,
        2.60023941e-01, 2.38596491e-01, 1.17025386e-01],
       [5.13720810e-01, 2.75514622e-04, 7.55980861e-01, ...,
        1.87533166e-01, 2.49122807e-01, 7.99220292e-02],
       [6.67181039e-01, 2.01856631e-03, 7.94258373e-01, ...,
        1.65553924e-01, 1.78947368e-01, 2.42004481e-02]])

# Pre-processing

Scale the data using the MinMaxScaler

In [9]:
#Checking the shapes
print(X_train.shape, y_train.shape)







(1173, 10) (1173,)


# Train the Support Vector Machine

In [0]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)

In [11]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.08996299, 0.09521674, 0.07388217, 0.06249424, 0.14425158,
       0.15965786, 0.08394282, 0.10320833, 0.09781644, 0.08956682])

In [12]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, df_X.columns), reverse=True)

[(0.15965785757574402, 'Fr'),
 (0.14425158450702594, 'Eng'),
 (0.10320832945457747,
  '    Average total income in 2015 among recipients ($)'),
 (0.09781644419888304, 'PovertyRate'),
 (0.09521674349687664, 'Population density per square kilometre'),
 (0.08996298796154238, 'Population, 2016'),
 (0.08956681818603832, 'Im'),
 (0.08394281902753264, 'Non-of'),
 (0.07388217468567762, 'Average age of the population'),
 (0.062494240906102005, 'Average household size')]

In [13]:
print(f"Training Data Score: {rf.score(X_train, y_train)}")
print(f"Testing Data Score: {rf.score(X_test, y_test)}")

Training Data Score: 0.7587382779198636
Testing Data Score: 0.6368286445012787


Using a support vector machine, linear kernel

In [0]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model2 = SVC(kernel='linear')
model2.fit(X_train, y_train)
predictions = model2.predict(X_test)


In [15]:
print(f"Training Data Score: {model2.score(X_train, y_train)}")
print(f"Testing Data Score: {model2.score(X_test, y_test)}")

Training Data Score: 0.5234441602728048
Testing Data Score: 0.5370843989769821


In [0]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [17]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    train_score = knn.score(X_train, y_train)
    test_score = knn.score(X_test, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")

k: 1, Train/Test Score: 0.685/0.565
k: 3, Train/Test Score: 0.728/0.621
k: 5, Train/Test Score: 0.715/0.591
k: 7, Train/Test Score: 0.665/0.558
k: 9, Train/Test Score: 0.641/0.558
k: 11, Train/Test Score: 0.616/0.555
k: 13, Train/Test Score: 0.605/0.558
k: 15, Train/Test Score: 0.576/0.532
k: 17, Train/Test Score: 0.573/0.537
k: 19, Train/Test Score: 0.558/0.532


In [18]:
# Note that k: 3 provides the best accuracy where the classifier starts to stablize
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
print('k=3 Test Acc: %.3f' % knn.score(X_test, y_test))

k=3 Test Acc: 0.621


In [19]:
df_test = pd.read_pickle("../ElecDistr2016Simp.pkl")
print(df_test.shape)
df_test.head()

(338, 11)


,GEO_ID,"Population, 2016",Population density per square kilometre,Average age of the population,Average household size,Eng,Fr,Non-of,Average total income in 2015 among recipients ($),PovertyRate,Im
0,10001,86494.0,13.4,42.4,2.4,0.992109,0.002280,0.005611,48083.0,9.3,0.011971
1,10002,74116.0,4.4,47.4,2.3,0.992590,0.002379,0.005031,38924.0,12.6,0.009546
2,10003,77680.0,2.0,46.5,2.3,0.991748,0.001949,0.006302,38258.0,15,0.009691
3,10004,27197.0,0.1,37.0,2.7,0.864492,0.015284,0.120224,56347.0,6.5,0.030252
4,10005,86553.0,2.3,46.5,2.3,0.986168,0.006595,0.007237,38151.0,14.9,0.014009


In [20]:
df_run = df_test.drop(columns = ["GEO_ID"])
df_run.head()



,"Population, 2016",Population density per square kilometre,Average age of the population,Average household size,Eng,Fr,Non-of,Average total income in 2015 among recipients ($),PovertyRate,Im
0,86494.0,13.4,42.4,2.4,0.992109,0.002280,0.005611,48083.0,9.3,0.011971
1,74116.0,4.4,47.4,2.3,0.992590,0.002379,0.005031,38924.0,12.6,0.009546
2,77680.0,2.0,46.5,2.3,0.991748,0.001949,0.006302,38258.0,15,0.009691
3,27197.0,0.1,37.0,2.7,0.864492,0.015284,0.120224,56347.0,6.5,0.030252
4,86553.0,2.3,46.5,2.3,0.986168,0.006595,0.007237,38151.0,14.9,0.014009


In [21]:
#Preprocessing done before the Train Test Split 

print(scaler)
scaler.fit(df_run)
X = scaler.transform(df_run)

print(X.shape)

MinMaxScaler(copy=True, feature_range=(0, 1))
(338, 10)


In [22]:


results2 = knn.predict(X)


print(len(results2))

counted = countResults(results2)

counted["Model"] = "Nearest Neighbor"
print(counted)

#forecast_df = pd.DataFrame.from_dict(counted, orient = 'index')

#forecast_df.head()


338
{'Liberal': 103, 'Conservative': 151, 'NDP': 34, 'Bloc': 49, 'Model': 'Nearest Neighbor'}


In [23]:
results3 = rf.predict(X)



counted1 = countResults(results3)

counted1["Model"] = "Random Forest"

#forecast_df = forecast_df.append(counted, ignore_index=True)
print(counted1)

#forecast_df.head()

{'Liberal': 112, 'Conservative': 135, 'NDP': 44, 'Bloc': 46, 'Model': 'Random Forest'}


In [24]:
results4 = model2.predict(X)

counted2 = countResults(results4)

counted2["Model"] = "SVM(linear)"

print(counted2)

{'Liberal': 67, 'Conservative': 196, 'NDP': 0, 'Bloc': 74, 'Model': 'SVM(linear)'}


In [25]:
listcount = [counted, counted1, counted2]

Liberal = []
Conservative = []
NDP = []
Bloc = []
Model = []
  

for count in listcount:
  Liberal.append(count.get('Liberal'))
  Conservative.append(count.get('Conservative'))
  NDP.append(count.get('NDP'))
  Bloc.append(count.get('Bloc'))
  Model.append(count.get("Model"))
  
  
print(Liberal, Bloc)

election_df = pd.DataFrame([counted, counted1, counted2])


election_df = election_df[["Model", "Liberal", "Conservative", "NDP", "Bloc"]]
  
election_df.head()

[103, 112, 67] [49, 46, 74]


,Model,Liberal,Conservative,NDP,Bloc
0,Nearest Neighbor,103,151,34,49
1,Random Forest,112,135,44,46
2,SVM(linear),67,196,0,74


In [0]:
election_df.to_pickle("ForecastCanada")